<a href="https://colab.research.google.com/github/iamharkirat/BMI598/blob/main/p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Conv1D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.regularizers import l2

# Load the data

In [111]:
df=pd.read_csv('/content/drive/MyDrive/bmi598/data2.csv')

In [112]:
df.head()

,Label,ax,ay,az,gx,gy,gz
0,Right,0.01,-1.01,0.07,0.31,-1.22,2.81
1,Right,0.03,-0.99,0.06,0.67,-0.98,2.50
2,Right,0.04,-0.98,0.06,0.98,-0.67,0.06
3,Right,0.02,-1.00,0.05,2.20,-0.79,-2.26
4,Right,0.00,-1.02,0.04,2.69,0.00,-2.50


## Evaluate the data

In [113]:
print(df.head())
print(df.info())
print(df['Label'].unique())
print(df['Label'].value_counts())


   Label    ax    ay    az    gx    gy    gz
0  Right  0.01 -1.01  0.07  0.31 -1.22  2.81
1  Right  0.03 -0.99  0.06  0.67 -0.98  2.50
2  Right  0.04 -0.98  0.06  0.98 -0.67  0.06
3  Right  0.02 -1.00  0.05  2.20 -0.79 -2.26
4  Right  0.00 -1.02  0.04  2.69  0.00 -2.50
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21400 entries, 0 to 21399
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Label   21400 non-null  object 
 1   ax      21400 non-null  float64
 2   ay      21400 non-null  float64
 3   az      21400 non-null  float64
 4   gx      21400 non-null  float64
 5   gy      21400 non-null  float64
 6   gz      21400 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.1+ MB
None
['Right' 'Supine' 'Left' 'Prone']
Supine    5572
Right     5443
Prone     5260
Left      5125
Name: Label, dtype: int64


In [114]:
df.isnull().sum()

Label    0
ax       0
ay       0
az       0
gx       0
gy       0
gz       0
dtype: int64

In [115]:
df = df.dropna()

## Sliding window function

In [116]:
def sliding_window(data, labels, window_size, step_size):
    num_windows = (data.shape[0] - window_size) // step_size + 1
    X = np.array([data[i:i+window_size] for i in range(0, num_windows * step_size, step_size)])

    # Assuming the label remains the same for the entire window (common for time-series classification tasks)
    y = np.array([labels[i] for i in range(0, num_windows * step_size, step_size)])

    return X, y


In [117]:
# Shuffle and balance each subset of labels separately
supine_df = df[df['Label'] == 'Supine'].sample(frac=1).reset_index(drop=True)
left_df = df[df['Label'] == 'Left'].sample(frac=1).reset_index(drop=True)
right_df = df[df['Label'] == 'Right'].sample(frac=1).reset_index(drop=True)
prone_df = df[df['Label'] == 'Prone'].sample(frac=1).reset_index(drop=True)

# Assuming your data is sampled at 50Hz
f = 50
window_size = 2*f
step_size = window_size

# Apply the sliding window function to each subset
X_supine, y_supine = sliding_window(supine_df.drop('Label', axis=1).values.astype(float), supine_df['Label'].values, window_size, step_size)
X_left, y_left = sliding_window(left_df.drop('Label', axis=1).values.astype(float), left_df['Label'].values, window_size, step_size)
X_right, y_right = sliding_window(right_df.drop('Label', axis=1).values.astype(float), right_df['Label'].values, window_size, step_size)
X_prone, y_prone = sliding_window(prone_df.drop('Label', axis=1).values.astype(float), prone_df['Label'].values, window_size, step_size)

# Combine the subsets
X_segments = np.concatenate([X_supine, X_left, X_right, X_prone])
y_segments = np.concatenate([y_supine, y_left, y_right, y_prone])

# Shuffle the entire dataset
indices = np.arange(X_segments.shape[0])
np.random.shuffle(indices)
X_segments = X_segments[indices]
y_segments = y_segments[indices]

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
y_segments = label_encoder.fit_transform(y_segments)

## Split data into train & test set

In [118]:
# Splitting dataset into Train and Test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_segments, y_segments, test_size=0.3, stratify=y_segments, random_state=42)

# Build the model

## conv1d

In [119]:
def enhanced_model_with_conv1d(activation_function='relu', input_shape=None):
    model = Sequential()

    # Hidden Layer 1
    model.add(Conv1D(512, kernel_size=3, activation=activation_function, kernel_regularizer=l2(0.001), input_shape=input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Hidden Layer 2
    model.add(Conv1D(256, kernel_size=3, activation=activation_function, kernel_regularizer=l2(0.001), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Hidden Layer 3
    model.add(Conv1D(128, kernel_size=3, activation=activation_function, kernel_regularizer=l2(0.001), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Hidden Layer 4
    model.add(Conv1D(64, kernel_size=3, activation=activation_function, kernel_regularizer=l2(0.001), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Flatten())  # Flatten the sequence data for dense output

    # Output Layer
    model.add(Dense(4, activation='softmax'))  # Assuming 4 classes

    return model

results_enhanced = {}
results_enhanced_train = {}  # To store training accuracies
reports = {}  # To store classification reports for test data
reports_train = {}  # To store classification reports for training data

for activation in ['sigmoid', 'tanh', 'relu']:
    model = enhanced_model_with_conv1d(activation, input_shape=(X_train.shape[1], X_train.shape[2]))
    optimizer = Adam(learning_rate=0.0001)

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))

    # Predictions and reports for the TEST set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    accuracy = accuracy_score(y_test, y_pred_classes)
    results_enhanced[activation] = accuracy * 100
    report = classification_report(y_test, y_pred_classes)
    reports[activation + "_enhanced"] = report

    # Predictions and reports for the TRAINING set
    y_pred_train = model.predict(X_train)
    y_pred_classes_train = np.argmax(y_pred_train, axis=1)

    accuracy_train = accuracy_score(y_train, y_pred_classes_train)
    results_enhanced_train[activation] = accuracy_train * 100
    report_train = classification_report(y_train, y_pred_classes_train)
    reports_train[activation + "_enhanced"] = report_train

# Print results
print("Enhanced Model with Conv1D Accuracy on TEST data:")
for activation, accuracy in results_enhanced.items():
    print(f"{activation.capitalize()}: {accuracy:.2f}%")

print("\nClassification Reports for Enhanced Model with Conv1D on TEST data:")
for activation, report in reports.items():
    if "_enhanced" in activation:
        print(f"\n{activation.capitalize()}:\n{report}")

print("\nEnhanced Model with Conv1D Accuracy on TRAINING data:")
for activation, accuracy in results_enhanced_train.items():
    print(f"{activation.capitalize()}: {accuracy:.2f}%")

print("\nClassification Reports for Enhanced Model with Conv1D on TRAINING data:")
for activation, report in reports_train.items():
    if "_enhanced" in activation:
        print(f"\n{activation.capitalize()}:\n{report}")


Epoch 1/200
3/3 [==============================] - 4s 194ms/step - loss: 2.8023 - accuracy: 0.2500 - val_loss: 2.3079 - val_accuracy: 0.2344
Epoch 2/200
3/3 [==============================] - 0s 26ms/step - loss: 2.5458 - accuracy: 0.2568 - val_loss: 2.2981 - val_accuracy: 0.2344
Epoch 3/200
3/3 [==============================] - 0s 32ms/step - loss: 2.5288 - accuracy: 0.3311 - val_loss: 2.2890 - val_accuracy: 0.2344
Epoch 4/200
3/3 [==============================] - 0s 32ms/step - loss: 2.3882 - accuracy: 0.3514 - val_loss: 2.2824 - val_accuracy: 0.2344
Epoch 5/200
3/3 [==============================] - 0s 25ms/step - loss: 2.4030 - accuracy: 0.3649 - val_loss: 2.2756 - val_accuracy: 0.2344
Epoch 6/200
3/3 [==============================] - 0s 34ms/step - loss: 2.1854 - accuracy: 0.3446 - val_loss: 2.2696 - val_accuracy: 0.2344
Epoch 7/200
3/3 [==============================] - 0s 33ms/step - loss: 1.9566 - accuracy: 0.5000 - val_loss: 2.2640 - val_accuracy: 0.2344
Epoch 8/200
3/3 [==

In [108]:
model.summary()

Model: "sequential_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_23 (Conv1D)          (None, 100, 512)          5120      
                                                                 
 batch_normalization_133 (B  (None, 100, 512)          2048      
 atchNormalization)                                              
                                                                 
 dropout_143 (Dropout)       (None, 100, 512)          0         
                                                                 
 conv1d_24 (Conv1D)          (None, 100, 256)          393472    
                                                                 
 batch_normalization_134 (B  (None, 100, 256)          1024      
 atchNormalization)                                              
                                                                 
 dropout_144 (Dropout)       (None, 100, 256)        